In [15]:
import weather_data as wd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import subprocess
import aiml
import os
import matplotlib.image as mpimg
import IPython
from IPython.display import IFrame

In [16]:
def wind(abbr):
    l1 = ["Noord", "NNO", "NO", "ONO", "Oost", "OZO", "ZO", "ZZO", "Zuid", "ZZW", "ZW", "WZW", "West", "WNW", "NW","NNW", "VAR"]
    l2 = ["noord", "noordnoordoost", "noordoost", "oostnoordoost", "oost", "oostzuidoost", "zuidoost", "zuidzuidoost", 
     "zuid", "zuidzuidwest", "zuidwest", "westzuidwest", "west", "westnoordwest", "noordwest", "noordnoordwest", "geen"]
    for i, j in enumerate(l1): 
        if abbr == j:
            return l2[i]
wind("NNO") 
    
def display_wind(winddir):
    l1 = ["N.png", "NNO.png", "NO.png", "ONO.png", "O.png", "OZO.png", "ZO.png", "ZZO.png", "Z.png", "ZZW.png", "ZW.png", "WZW.png", "W.png", "WNW.png", "NW.png","NNW.png", "VAR.png"]
    l2 = ["noord", "noordnoordoost", "noordoost", "oostnoordoost", "oost", "oostzuidoost", "zuidoost", "zuidzuidoost", 
     "zuid", "zuidzuidwest", "zuidwest", "westzuidwest", "west", "westnoordwest", "noordwest", "noordnoordwest", "geen"]
    for i, j in enumerate(l2):
        if winddir == j:
            return(mpimg.imread('images/'+l1[i]))

In [20]:
# Create the kernel and learn AIML files
kernel = aiml.Kernel()
kernel.learn("start_bot.xml")
kernel.respond("load aiml")

kernel.setBotPredicate("name", "WeatherBot")

# Press CTRL-C to break this loop
while True:
    message = input("Enter your message >> ")
    if message == "quit":
        break #useful in notebook
        # exit() # useful in actual application
    elif message == "save":
        kernel.saveBrain("aiml/bot_brain.brn")
    else:
        bot_response = kernel.respond(message)
        if kernel.getPredicate("graph_temp") == "1":
            tmax_0 = int(kernel.getPredicate("d0tmax"))
            tmax_1 = int(kernel.getPredicate("d1tmax"))
            tmax_2 = int(kernel.getPredicate("d2tmax"))
            tmin_0 = int(kernel.getPredicate("d0tmin"))
            tmin_1 = int(kernel.getPredicate("d1tmin"))
            tmin_2 = int(kernel.getPredicate("d2tmin"))
            tmax = [tmax_0, tmax_1, tmax_2]
            tmin = [tmin_0, tmin_1, tmin_2]
            days = ["vandaag", "morgen", "overmorgen"]
            l2 = plt.plot(days, tmin, color = "blue", label="minimum temperatuur")
            l1 = plt.plot(days, tmax, color = "orange", label="maximum temperatuur")
            plt.xlabel("Tijdstip")
            plt.ylabel("Temperatuur")
            plt.legend()
            plt.show()
            kernel.setPredicate("graph_temp", "0")
            print(kernel.respond('SPECIFICS'))
            
        elif kernel.getPredicate("graph_temp") == "2":
            fig = plt.figure()
            fig.set_figheight(15)
            fig.set_figwidth(15)
            for i in ["d0", "d1", "d2"]:
                ax = fig.add_subplot(1,3,int(i[1])+1)
                plt.xticks([])
                plt.yticks([])
                plt.title("Dag: "+i[1] +", Windkracht: "+ kernel.getPredicate(i+"windk"))
                plt.imshow(display_wind(kernel.getPredicate(i+"windr")))
            plt.show()
            kernel.setPredicate("graph_temp", "0")
            print(kernel.respond('SPECIFICS'))
            
        elif kernel.getPredicate("graph_temp") == "3":
            graph_avail_stations = [6391, 6275, 6249, 6260, 6235, 6370, 6377, 6350, 6323, 6280, 6330, 6279,
                                    6251, 6225, 6270, 6269, 6348, 6380, 6273, 6344, 6240, 6267, 6290, 6242,
                                    6310, 6375, 6319, 6340]
            station_id = wd.weather_data(kernel.getPredicate("cur_weather_loc"))['stationid']
            if station_id not in graph_avail_stations:
                station_id = min(graph_avail_stations, key=lambda x:abs(x-station_id))
            iframe = '//gadgets.buienradar.nl/gadget/forecastandstation/{}/'.format(station_id)
            IPython.display.display(IFrame(iframe, width=700, height=350))
            kernel.setPredicate("graph_temp", "0")
            print(kernel.respond('SPECIFICS'))
        
        elif kernel.getPredicate("flag") == "2":
            t = int(round(float(kernel.getPredicate("gtemp"))))
            if t < 5:
                if float(kernel.getPredicate("d0neerslag")) < 30:
                    kernel.getPredicate("flag") == "0"
                    print(kernel.respond('COLD DRY'))
                else:
                    kernel.getPredicate("flag") == "0"
                    print(kernel.respond('COLD DRY'))
            elif t < 15:
                if float(kernel.getPredicate("d0neerslag")) < 30:
                    kernel.getPredicate("flag") == "0"
                    print(kernel.respond('MIDDLE DRY'))
                else:
                    kernel.getPredicate("flag") == "0"
                    print(kernel.respond('MIDDLE RAIN'))
            else:
                if float(kernel.getPredicate("d0neerslag")) < 30:
                    kernel.getPredicate("flag") == "0"
                    print(kernel.respond('WARM DRY'))
                else:
                    kernel.getPredicate("flag") == "0"
                    print(kernel.respond('WARM RAIN'))
            kernel.setPredicate("flag", "0")
            print(kernel.respond('SPECIFICS'))
            
        elif kernel.getPredicate("flag") == "1":
            weather_dict = wd.weather_data(kernel.getPredicate("cur_weather_loc"))
            kernel.setPredicate("weather_descr", weather_dict["samenv"])
            kernel.setPredicate("temp", weather_dict["temp"])
            kernel.setPredicate("weather_forecast", weather_dict["verw"])
            print(kernel.respond('CURRENT WEATHER LOC'))
            print(kernel.respond('SPECIFICS'))
            for i in weather_dict:
                if "windr" in i: 
                    kernel.setPredicate(i, wind(weather_dict[i]))
                else:
                    kernel.setPredicate(i, weather_dict[i])
            kernel.setPredicate("flag", "0")
        else:
            print(bot_response)

Loading start_bot.xml...done (0.00 seconds)
Loading aiml/basic_chat.aiml...done (0.00 seconds)
Loading aiml/weather_chat.aiml...done (0.00 seconds)
Loading aiml/specifics.aiml...done (0.00 seconds)
Loading aiml/exceptions.aiml...done (0.01 seconds)
Enter your message >> hoe is het weer in blokker
Momenteel is het geheel bewolkt en 8.5 °C in blokker.
Wilt u meer weten over het weer in blokker?
Enter your message >> ja
Waarover wilt u meer informatie?
Enter your message >> kleding
Het is aardig weer buiten, niet te warm en ook niet te koud en niet veel regen. Trek een jasje en een lange broek aan.
Wilt u meer weten over het weer in blokker?
Enter your message >> quit


In [124]:
wd.weather_data("Vlissingen")

{'plaats': 'Westkapelle',
 'temp': '8.9',
 'gtemp': '5.2',
 'samenv': 'Geheel bewolkt',
 'lv': '68',
 'windr': 'West',
 'windms': '8',
 'winds': '5',
 'windk': '15.6',
 'windkmh': '28.8',
 'luchtd': '1013.0',
 'ldmmhg': '760',
 'dauwp': '2',
 'zicht': '24',
 'verw': 'Wolkenvelden en vooral vanavond en vannacht enkele buien',
 'sup': '07:14',
 'sunder': '20:23',
 'image': 'bewolkt',
 'd0weer': 'zonnig',
 'd0tmax': '12',
 'd0tmin': '4',
 'd0windk': '3',
 'd0windknp': '8',
 'd0windms': '4',
 'd0windkmh': '15',
 'd0windr': 'NO',
 'd0neerslag': '0',
 'd0zon': '94',
 'd1weer': 'bewolkt',
 'd1tmax': '9',
 'd1tmin': '3',
 'd1windk': '3',
 'd1windknp': '8',
 'd1windms': '4',
 'd1windkmh': '15',
 'd1windr': 'NW',
 'd1neerslag': '30',
 'd1zon': '30',
 'd2weer': 'halfbewolkt',
 'd2tmax': '9',
 'd2tmin': '-2',
 'd2windk': '2',
 'd2windknp': '6',
 'd2windms': '3',
 'd2windkmh': '11',
 'd2windr': 'ZO',
 'd2neerslag': '20',
 'd2zon': '40',
 'alarm': '0',
 '$id': '7',
 'stationid': 6308,
 'stationname'

In [133]:
graph_avail_stations = [6391, 6275, 6249, 6260, 6235, 6370, 6377, 6350, 6323, 6280, 6330, 6279, 6251, 6225, 6270, 6269, 6348, 6380, 6273, 6344, 6240, 6267, 6290, 6242, 6310, 6375, 6319, 6340]
station_id = wd.weather_data("Hoorn")['stationid']

if station_id not in graph_avail_stations:
    station_id = min(graph_avail_stations, key=lambda x:abs(x-station_id))
    
iframe = '//gadgets.buienradar.nl/gadget/forecastandstation/{}/'.format(station_id)
IPython.display.display(IFrame(iframe, width=700, height=350))

